In [2]:
!pip install dash

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/00/69/d0ee9c6524678e98d336464d5c898182794c6b2ba1e5507bc7010a126ce2/dash-2.14.1-py3-none-any.whl.metadata
  Obtaining dependency information for Flask<3.1,>=1.0.4 from https://files.pythonhosted.org/packages/36/42/015c23096649b908c809c69388a805a571a3bea44362fe87e33fc3afa01f/flask-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for Werkzeug<3.1 from https://files.pythonhosted.org/packages/c3/fc/254c3e9b5feb89ff5b9076a23218dafbc99c96ac5941e900b71206e6313b/werkzeug-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.6.2 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.6/10.4 MB 19.5 MB/s eta 0:00:01
   ------ ------------------------------

In [5]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Output, Input, State, callback, no_update
import dash_core_components as dcc
import pandas as pd
import json

app = dash.Dash()

df = pd.read_csv('estrategis-ec.com_data_dash_data.csv',delimiter =";")

def fig_map(df):
    fig = px.scatter_mapbox(df, lat="lat", lon="lon",
                            height=600, width=900,
                            color="Empresas", size="Empresas", size_max=75, animation_frame='year',
                            color_continuous_scale='viridis',
                            mapbox_style="carto-positron", zoom=5.5, hover_name="City",
                            center={"lat": -2, "lon": -79.01})

    return fig

app.layout = html.Div([

    html.Div([
        dcc.Graph(id='map_plot1', figure=fig_map(df))],
        style={'width': '75%', 'display': 'inline-block'}),

    html.Div(style={'width': '25%', 'display': 'inline-block'}, id='my-graph-container')
])


##calback
@callback(
    Output(component_id='my-graph-container', component_property='children'),
    Input(component_id='map_plot1', component_property='clickData'))
def update_graph(clickData):
    if clickData:
        city = clickData['points'][0]['hovertext']
        print(city)

        dff = df[df['City'] == city]
        print(dff.head())
        fig1 = px.line(dff, x='year', y='Empresas', title=city)
        return dcc.Graph(figure=fig1)
    else:
        no_update


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8059)